# Keras Tuner
- 최적의 Hyperparameter 세트 선택에 도움을 주는 라이이브러리
- Hyperparameter는 훈련 프로세스 및 모델의 토폴로지를 제어하는 변수
- Hyperparameter의 두 종류
    1. 숨겨진 레이어 수 및 너비와 같이 모델 선택에 영향을 주는 모델 하이퍼파라미터
    2. SDG(Stochastic Gradient Descent)의 학습률 및 KNN(K Nearest Neighbors) 분류자의 최근접 이웃 수와 같은 학습 알고리즘의 속도와 품질에 영향을 주는 알고리즘 하이퍼파라미터
    
---    
    
### 설정

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

- keras tuner 설치 및 import

In [2]:
!pip install -q -U keras-tuner
import keras_tuner as kt

---

### 데이터세트 다운로드 및 준비
- Fashion MNIST를 대상으로 최적 하이퍼파라미터 찾기

In [3]:
# 데이터 로드
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

---

### 모델 정의하기
- 하이퍼튜닝을 위한 모델(하이퍼 모델)을 빌드할 때는 모델 아키텍처와 더불어 하이퍼파라미터 검색 공간도 정의
- 하이퍼 모델을 정의하는 두 가지 접근 방법
    1. 모델 빌더 함수 사용
    2. Keras Tuner API의 HyperModel 클래스를 상속

- 두 개의 사전 정의된 HyperModel인 HyperXception과 HyperResNet을 사용 가능하다.
- 모델 빌더를 사용하여 모델 정의

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

---

### 튜너를 인스턴스화 하고 하이퍼튜닝 수행하기
- Keras Tuner의 4가지 튜너
    - RandomSearch
    - Hyperband
    - BayesianOptimization
    - Sklearn
- Hyperband 사용 시 최적화 할 하이퍼모델인 objective와 훈현할 최대 epoch 수(max_epochs)를 지정해야 한다.
- Hyperband 튜닝 알고리즘은 적응형 리소스 할당 및 조기중단을 사용하여 고성능 모델에서 신속하게 수렴한다.

> 스포츠 챔피언십 스타일 브래킷 :  
> 몇 개의 epoch에 대해 많은 수의 모델을 훈련하고 최고 성능을 보이는 절반만 다음 단계로 넘깁니다.  
> Hyperband는 1 + log<sub>factor</sub>( max_epochs)를 계산하고 이를 가장 가까운 정수로 반올림하여  
> 한 브래킷에서 훈련할 모델 수를 결정

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
Metal device set to: Apple M1
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


2022-04-06 11:16:18.976443: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-06 11:16:18.976599: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


- 하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

- 하이퍼파라미터 검색을 실행. 검색 메서드의 인수는 위의 콜백 외에 tf.keras.model.fit에 사용되는 인수와 같다.

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 16s]
val_accuracy: 0.8568000197410583

Best val_accuracy So Far: 0.8844000697135925
Total elapsed time: 00h 03m 52s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



- 최적의 파라미터로 모델을 재훈련

In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
  28/1875 [..............................] - ETA: 6s - loss: 1.1810 - accuracy: 0.6038

2022-04-06 13:01:26.549258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1867/1875 [============================>.] - ETA: 0s - loss: 0.4797 - accuracy: 0.8301

2022-04-06 13:01:33.487759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 8s 4ms/step - loss: 0.4797 - accuracy: 0.8302 - val_loss: 0.4088 - val_accuracy: 0.8507
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3600 - accuracy: 0.8691 - val_loss: 0.3960 - val_accuracy: 0.8572
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3236 - accuracy: 0.8802 - val_loss: 0.3817 - val_accuracy: 0.8639
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2986 - accuracy: 0.8898 - val_loss: 0.3820 - val_accuracy: 0.8655
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2807 - accuracy: 0.8961 - val_loss: 0.3409 - val_accuracy: 0.8764
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2658 - accuracy: 0.9005 - val_loss: 0.3376 - val_accuracy: 0.8792
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2539 - accuracy: 0.9058 - val_loss: 0.3565 - val_accuracy: 0.8725
Epo

---

### 요약
- [TensorFlow 블로그의 Keras Tuner](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Keras Tuner 웹 사이트](https://keras-team.github.io/keras-tuner/)